# ML - lr

## dataset construct

In [2]:
from pyspark.sql.functions import *

In [3]:
d = spark.read.csv('/user/maxnet/database/sig.db/data_ct_res_hostname/*',sep='\x01')
#d.show(3,truncate=False)

In [4]:
d = d.withColumnRenamed('_c1','val').withColumnRenamed('_c3','osid')
#d.show(truncate=False)

In [5]:
d.select('val','osid').count()

434021295

In [6]:
d = d.select('val','osid').dropna().distinct()

In [7]:
d.count()

131947294

In [12]:
d1 = d.filter(d.osid == 0).dropna().distinct().withColumnRenamed('osid','label').select('val',col('label').cast('float'))
d2 = d.filter(d.osid != 0).dropna().distinct().withColumn('label',lit(1)).select('val',col('label').cast('float'))

In [13]:
d1.count(),d1.distinct().count(),d2.count(),d2.distinct().count()

(13366199, 13366199, 118581095, 118581095)

In [15]:
13366199/118581095

0.11271779030207134

In [16]:
d2 = d2.sample(0.2)
d2.count()

23706806

## baseline accuracy

In [18]:
23706806/(13366199+23706806)

0.6394627573351553

In [19]:
23706806+13366199

37073005

In [20]:
df = d1.union(d2)
df.count()

37073005

In [26]:
df.count()

37073005

In [37]:
df.filter(df.label == 1).count(),df.filter(df.label == 0).count()

(23706806, 13366199)

In [38]:
from pyspark.ml.feature import RegexTokenizer
tokenizer = RegexTokenizer(inputCol='val',outputCol='split',pattern='\\W')
splitdf = tokenizer.transform(df)
#splitdf.show(5,truncate=False)

In [21]:
#splitdf.sample(0.001).show(20,truncate=False)

+--------------------------------+-----+----------------------------------+
|val                             |label|split                             |
+--------------------------------+-----+----------------------------------+
|9F1004990070344000000495739871E |0.0  |[9f1004990070344000000495739871e] |
|CF1001990070114000034C09B4B838AB|0.0  |[cf1001990070114000034c09b4b838ab]|
|C49266940                       |0.0  |[c49266940]                       |
|DEEP-20130415LL                 |0.0  |[deep, 20130415ll]                |
|GHG-20190104LOF                 |0.0  |[ghg, 20190104lof]                |
|menpeki                         |0.0  |[menpeki]                         |
|CSF8LOLZZ51SDB0                 |0.0  |[csf8lolzz51sdb0]                 |
|CA100499007034400000D071C4DA2889|0.0  |[ca100499007034400000d071c4da2889]|
|FHYZERTL5ABHVL5                 |0.0  |[fhyzertl5abhvl5]                 |
|EJJ66HW7FSALSRX                 |0.0  |[ejj66hw7fsalsrx]                 |
|Lyyyxin    

In [39]:
from pyspark.ml.feature import CountVectorizer

count = CountVectorizer(inputCol='split',outputCol='rawFeatures')
model = count.fit(splitdf)
featuredf = model.transform(splitdf)
#featuredf.sample(0.0011).show(20,truncate=False)

In [40]:
from pyspark.ml.feature import IDF

idf = IDF(inputCol='rawFeatures',outputCol='features')
idfModel = idf.fit(featuredf)
finaldf = idfModel.transform(featuredf)
fdf = finaldf.select('label','features')


In [23]:
fdf.show(5,truncate=False)

+-----+----------------------------------+
|label|features                          |
+-----+----------------------------------+
|0.0  |(262144,[40],[6.068304290554965]) |
|0.0  |(262144,[1071],[10.6055264499684])|
|0.0  |(262144,[],[])                    |
|0.0  |(262144,[],[])                    |
|0.0  |(262144,[],[])                    |
+-----+----------------------------------+
only showing top 5 rows



In [41]:
seed = 0 
trainDF, testDF = fdf.randomSplit([0.7,0.3],seed)

In [42]:
trainDF.count(),testDF.count()

(25952091, 11120914)

In [43]:
25952091/(25952091+11120914)

0.7000266366322342

In [45]:
trainDF.describe('label').show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|           25952091|
|   mean| 0.6394275898616416|
| stddev|0.48016659199639733|
|    min|                0.0|
|    max|                1.0|
+-------+-------------------+



In [46]:
testDF.describe('label').show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|           11120914|
|   mean| 0.6395448251825344|
| stddev|0.48013254679717454|
|    min|                0.0|
|    max|                1.0|
+-------+-------------------+



In [47]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator,ParamGridBuilder
import numpy as np

lr = LogisticRegression(maxIter=10)

paramGrid_lr = ParamGridBuilder()\
       .addGrid(lr.regParam,np.linspace(0.3,0.01,10))\
       .addGrid(lr.elasticNetParam,np.linspace(0.3,0.8,6))\
       .build()

crossval_lr = CrossValidator(estimator=lr,
                            estimatorParamMaps=paramGrid_lr,
                            evaluator=BinaryClassificationEvaluator(),
                            numFolds=5)

cvModel_lr = crossval_lr.fit(trainDF)
best_model_lr = cvModel_lr.bestModel.summary
best_model_lr.predictions.columns

['label', 'features', 'rawPrediction', 'probability', 'prediction']

In [48]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
my_eval_lr = BinaryClassificationEvaluator(
    rawPredictionCol='prediction', labelCol='label', metricName='areaUnderROC')
my_eval_lr.evaluate(best_model_lr.predictions)

0.9558712944229603

In [49]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
my_mc_lr = MulticlassClassificationEvaluator(
    predictionCol='prediction', labelCol='label', metricName='f1')
my_mc_lr.evaluate(best_model_lr.predictions)

0.9516641131287215

In [50]:
my_mc_lr = MulticlassClassificationEvaluator(
    predictionCol='prediction', labelCol='label', metricName='accuracy')
my_mc_lr.evaluate(best_model_lr.predictions)

0.9512409231302402

In [51]:
train_fit_lr = best_model_lr.predictions.select('label', 'prediction')
train_fit_lr.groupBy('label', 'prediction').count().show()

+-----+----------+--------+
|label|prediction|   count|
+-----+----------+--------+
|  1.0|       1.0|15588083|
|  0.0|       1.0|  257814|
|  1.0|       0.0| 1006400|
|  0.0|       0.0| 9099794|
+-----+----------+--------+



In [81]:
(15588083+9099794)/(15588083+257814+1006400+9099794)

0.9512866227233867

In [52]:
predictions_lr = cvModel_lr.transform(testDF)
predictions_lr.columns

['label', 'features', 'rawPrediction', 'probability', 'prediction']

In [53]:
predictions_lr.show(5)

+-----+--------------+--------------------+--------------------+----------+
|label|      features|       rawPrediction|         probability|prediction|
+-----+--------------+--------------------+--------------------+----------+
|  0.0|(262144,[],[])|[1.48859558637857...|[0.81586738420457...|       0.0|
|  0.0|(262144,[],[])|[1.48859558637857...|[0.81586738420457...|       0.0|
|  0.0|(262144,[],[])|[1.48859558637857...|[0.81586738420457...|       0.0|
|  0.0|(262144,[],[])|[1.48859558637857...|[0.81586738420457...|       0.0|
|  0.0|(262144,[],[])|[1.48859558637857...|[0.81586738420457...|       0.0|
+-----+--------------+--------------------+--------------------+----------+
only showing top 5 rows



In [54]:
predictions_lr.select('label', 'prediction').show(20)

+-----+----------+
|label|prediction|
+-----+----------+
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
+-----+----------+
only showing top 20 rows



In [55]:
predictions_lr.groupBy('label', 'prediction').count().show()

+-----+----------+-------+
|label|prediction|  count|
+-----+----------+-------+
|  1.0|       0.0| 432142|
|  0.0|       0.0|3897988|
|  1.0|       1.0|6680181|
|  0.0|       1.0| 110603|
+-----+----------+-------+



# Accuracy with the test data

In [68]:
(3897988+6680181)/(432142+3897988+6680181+110603)

0.9511960078101495

# Out of Model Test

In [57]:
dtest = d2.sample(0.02)

In [58]:
dtest.count()

474233

In [59]:
dtest_split = tokenizer.transform(dtest)

In [60]:
count_test = CountVectorizer(inputCol='split',outputCol='rawFeatures')
model_test = count_test.fit(dtest_split)
featuredf_test = model_test.transform(dtest_split)
featuredf_test.sample(0.01).show(20,truncate=False)

+-------------------------+-----+-----------------------------+-------------------------+
|val                      |label|split                        |rawFeatures              |
+-------------------------+-----+-----------------------------+-------------------------+
|android-b55977fd05a44a1f |1.0  |[android, b55977fd05a44a1f]  |(262144,[0],[1.0])       |
|android-4a48e8160d8baa75 |1.0  |[android, 4a48e8160d8baa75]  |(262144,[0],[1.0])       |
|android-5f8d199d26e315bc |1.0  |[android, 5f8d199d26e315bc]  |(262144,[0],[1.0])       |
|android-898a098a0e060d28 |1.0  |[android, 898a098a0e060d28]  |(262144,[0],[1.0])       |
|android-c722e3460141f20c |1.0  |[android, c722e3460141f20c]  |(262144,[0],[1.0])       |
|android-ae068b05d32c15cb |1.0  |[android, ae068b05d32c15cb]  |(262144,[0],[1.0])       |
|litokudpleWatch          |1.0  |[litokudplewatch]            |(262144,[],[])           |
|android-6b3fcfcbbe6787da |1.0  |[android, 6b3fcfcbbe6787da]  |(262144,[0],[1.0])       |
|DESKTOP-U

In [61]:
idf = IDF(inputCol='rawFeatures',outputCol='features')
idfModel_test = idf.fit(featuredf_test)
finaldf_test = idfModel_test.transform(featuredf_test)
fdf_test = finaldf_test.select('label','features')

In [62]:
predictions_lr = cvModel_lr.transform(fdf_test)
predictions_lr.columns

['label', 'features', 'rawPrediction', 'probability', 'prediction']

In [63]:
predictions_lr.show(5)

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  1.0|(262144,[0],[0.54...|[-1.3145587763647...|[0.21172499575680...|       1.0|
|  1.0|(262144,[47],[5.8...|[1.79994724901845...|[0.85814251363601...|       0.0|
|  1.0|      (262144,[],[])|[1.48859558637857...|[0.81586738420457...|       0.0|
|  1.0|(262144,[42],[5.7...|[2.79417853498182...|[0.94236043122280...|       0.0|
|  1.0|(262144,[0],[0.54...|[-1.3145587763647...|[0.21172499575680...|       1.0|
+-----+--------------------+--------------------+--------------------+----------+
only showing top 5 rows



In [64]:
predictions_lr.select('label', 'prediction').show(20)

+-----+----------+
|label|prediction|
+-----+----------+
|  1.0|       0.0|
|  1.0|       0.0|
|  1.0|       1.0|
|  1.0|       0.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       0.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
+-----+----------+
only showing top 20 rows



In [65]:
predictions_lr.groupBy('label', 'prediction').count().show()

+-----+----------+------+
|label|prediction| count|
+-----+----------+------+
|  1.0|       0.0| 56662|
|  1.0|       1.0|417571|
+-----+----------+------+



In [83]:
417571/(56662+417571)

0.8805186480063597

In [66]:
my_mc_lr = MulticlassClassificationEvaluator(
    predictionCol='prediction', labelCol='label', metricName='accuracy')
my_mc_lr.evaluate(predictions_lr)

0.8804596052995046

# Unknown Test

In [278]:
un = spark.read.csv('/user/maxnet/database/sig.db/data_visual_unknown/*',sep='\x01')
un1 = un.sample(0.1).select('_c2').withColumnRenamed('_c2','val').distinct().dropna()
un1.count()

687

In [212]:
un_split = tokenizer.transform(un1)

In [213]:
count_un = CountVectorizer(inputCol='split',outputCol='rawFeatures')
model_un = count_un.fit(un_split)
featuredf_un = model_un.transform(un_split)
featuredf_un.sample(0.01).show(20,truncate=False)

+-------------------------------+-------------------------------------+-----------------------------------------------------+
|val                            |split                                |rawFeatures                                          |
+-------------------------------+-------------------------------------+-----------------------------------------------------+
|0d477ba7380bdce1d4760ae5ff3c6db|[0d477ba7380bdce1d4760ae5ff3c6db]    |(262144,[],[])                                       |
|0c4db2da22ffd24e50f39fa3147cb25|[0c4db2da22ffd24e50f39fa3147cb25]    |(262144,[178245],[1.0])                              |
|VGOPPP3ZHRENYWH                |[vgoppp3zhrenywh]                    |(262144,[],[])                                       |
|uplus-haier-0202-87e9-v5-sapkz |[uplus, haier, 0202, 87e9, v5, sapkz]|(262144,[41,44,45,46,77],[1.0,1.0,1.0,1.0,1.0])      |
|ZTE8c:68:c8:bd:51:a4           |[zte8c, 68, c8, bd, 51, a4]          |(262144,[1,2,3,50,146,314],[1.0,1.0,1.0,1.0,1.0

In [214]:
featuredf_un.columns

['val', 'split', 'rawFeatures']

In [215]:
idf_un = IDF(inputCol='rawFeatures',outputCol='features')
idfModel_un = idf_un.fit(featuredf_un)
finaldf_un = idfModel_un.transform(featuredf_un)
fdf_un = finaldf_un.select('val','features')

In [216]:
predictions_lr = cvModel_lr.transform(fdf_un)
predictions_lr.columns

['val', 'features', 'rawPrediction', 'probability', 'prediction']

In [217]:
predictions_lr.groupby('prediction').count().show()

+----------+------+
|prediction| count|
+----------+------+
|       0.0|316712|
|       1.0| 50414|
+----------+------+



In [218]:
50414/(316712+50414)

0.13732070188436668

In [219]:
from pyspark.sql.types import DoubleType
unlist = udf(lambda x: float(list(x)[1]), DoubleType())

In [220]:
target = predictions_lr.select('val',unlist('probability').alias('probability'),'prediction').filter(predictions_lr.prediction == 1).filter(predictions_lr.val != 'unknown').filter(predictions_lr.val != 'empty').filter(predictions_lr.val != 'NONE').filter(predictions_lr.val != 'none').filter(predictions_lr.val != 'N/A').filter(predictions_lr.val != 'normal').filter(predictions_lr.val != 'anonymous').filter(predictions_lr.val != 'null')

In [221]:
target.sample(0.001).show(100,truncate=False)

+---------------------+------------------+----------+
|val                  |probability       |prediction|
+---------------------+------------------+----------+
|jerry-PC             |0.9999999838413212|1.0       |
|regong-PC            |0.9999999838413212|1.0       |
|ZTE30:99:35:bb:e5:64 |0.999800297678007 |1.0       |
|Dsj-pc               |0.9999999838413212|1.0       |
|HSX-PC               |0.9999999838413212|1.0       |
|OY-05                |0.7271396868092981|1.0       |
|ai-PC                |0.9999999838413212|1.0       |
|VIP-PC               |0.9999999838413212|1.0       |
|AUTOBVT-KS60E4O      |0.9140328288996845|1.0       |
|ZTEb0:ac:d2:0b:07:d4 |0.998834672886984 |1.0       |
|ZTE24:58:6e:84:bc:40 |0.9999999957507975|1.0       |
|AUTOBVT-86CI4L9      |0.9140328288996845|1.0       |
|ZTEb0:ac:d2:0e:83:7e |0.9999997019069204|1.0       |
|YynnG-2              |0.9161485791529314|1.0       |
|BF-20180316RPDH      |0.8920025591107228|1.0       |
|JCG-55F86A           |0.609

In [227]:
d2.filter(d2.val.contains('ZTE')).show(truncate=False)


+----------------------+-----+
|val                   |label|
+----------------------+-----+
|PC-20180823OZTE       |1.0  |
|PC-20160106ZTEB       |1.0  |
|PC-20190110ZTEZ       |1.0  |
|PC-20181202ZTET       |1.0  |
|ZTE-Router-ee:73:a6   |1.0  |
|XP-20150528AZTE       |1.0  |
|PC-20151128ZTEH       |1.0  |
|ZTE-Router-ee:71:c6   |1.0  |
|ZTEQ802T-Android      |1.0  |
|PC-20171107XZTE       |1.0  |
|PC-20180311TZTE       |1.0  |
|PC-20181022ZTEK       |1.0  |
|PC-20180417RZTE       |1.0  |
|XP-20170810ZTEB       |1.0  |
|PC-20160226IZTE       |1.0  |
|ZTEGrandSIILTE-ZTES291|1.0  |
|ZTE-Router-ee:8a:14   |1.0  |
|ZTE-V1000             |1.0  |
|XP-20150419ZTEE       |1.0  |
|PC-20190314ZTEZ       |1.0  |
+----------------------+-----+
only showing top 20 rows



In [228]:
d1.filter(d1.val.contains('ZTE')).show(truncate=False)

+--------------------+-----+
|val                 |label|
+--------------------+-----+
|ZTE9c:6f:52:3e:e4:71|0.0  |
|ZTE9c:6f:52:3f:01:15|0.0  |
|ZTE9c:6f:52:3f:09:ed|0.0  |
|ZTE9c:6f:52:3f:0c:07|0.0  |
|ZTE9c:6f:52:3f:18:1d|0.0  |
|ZTE9c:6f:52:3f:2c:e1|0.0  |
|ZTE9c:6f:52:3f:46:31|0.0  |
|ZTE9c:6f:52:3f:4f:63|0.0  |
|ZTE9c:6f:52:3f:62:fd|0.0  |
|ZTE9c:6f:52:3f:89:eb|0.0  |
|ZTE9c:6f:52:3f:92:e3|0.0  |
|ZTE7c:39:53:83:5f:f0|0.0  |
|ZTE7c:39:53:83:66:84|0.0  |
|ZTE7c:39:53:83:7e:8e|0.0  |
|ZTE7c:39:53:83:82:e4|0.0  |
|ZTE7c:39:53:83:93:c6|0.0  |
|ZTE7c:39:53:83:96:ba|0.0  |
|ZTE7c:39:53:83:b6:56|0.0  |
|ZTE7c:39:53:83:c4:14|0.0  |
|ZTE7c:39:53:83:c9:34|0.0  |
+--------------------+-----+
only showing top 20 rows



In [225]:
target.sort('probability',ascending=False).show(100,truncate=False)

+---------------------+------------------+----------+
|val                  |probability       |prediction|
+---------------------+------------------+----------+
|rf2018-9-1-2-PC      |0.9999999999749836|1.0       |
|admin-01-PC          |0.9999999999463685|1.0       |
|HP SlateBook 10 x2 PC|0.9999999999408862|1.0       |
|lf-2018-1-14-PC      |0.999999999923211 |1.0       |
|1-1-PC               |0.9999999999092317|1.0       |
|new-2-PC             |0.9999999999053006|1.0       |
|2014-10-14-PC        |0.9999999998744782|1.0       |
|HS-39-PC             |0.9999999998410836|1.0       |
|84-PC                |0.9999999997088556|1.0       |
|XZ-PC                |0.9999999997015532|1.0       |
|xz-PC                |0.9999999997015532|1.0       |
|dc-PC                |0.9999999996934028|1.0       |
|DC-PC                |0.9999999996934028|1.0       |
|BC-BGS-PC            |0.9999999996907742|1.0       |
|bl-PC                |0.9999999996871056|1.0       |
|ZTE8c:68:c8:bc:c8:bc |0.999

In [195]:
save model:
#cvModel_lr.save('hdfs:///data/user/hive/warehouse/ian/model/model1')

load model:
#xx = cvModel_lr.load('hdfs:///data/user/hive/warehouse/ian/model/model1')

In [201]:
save best_model:
#cvModel_lr.bestModel.load('hdfs:///data/user/hive/warehouse/ian/model/model2')

load best_model:
#xxx = cvModel_lr.bestModel.load('hdfs:///data/user/hive/warehouse/ian/model/model2')

In [202]:
xxx_predictions = xxx.transform(fdf_un)
tar = xx_predictions.select('val',unlist('probability').alias('probability'),'prediction').filter(xx_predictions.prediction == 1).filter(xx_predictions.val != 'unknown').filter(xx_predictions.val != 'empty').filter(xx_predictions.val != 'NONE').filter(xx_predictions.val != 'none').filter(xx_predictions.val != 'N/A').filter(xx_predictions.val != 'normal').filter(xx_predictions.val != 'anonymous').filter(xx_predictions.val != 'null')
tar.sample(0.001).show(100,truncate=False)

+---------------------------+------------------+----------+
|val                        |probability       |prediction|
+---------------------------+------------------+----------+
|Windows 98                 |0.993063798390144 |1.0       |
|Windows 98                 |0.993063798390144 |1.0       |
|Windows 98                 |0.993063798390144 |1.0       |
|Windows 98                 |0.993063798390144 |1.0       |
|V1818CA                    |0.684273528029555 |1.0       |
|Windows 98                 |0.993063798390144 |1.0       |
|Windows NT 6.1             |0.9999992457843718|1.0       |
|Windows 98                 |0.993063798390144 |1.0       |
|Windows 98                 |0.993063798390144 |1.0       |
|KSA-AL00-1a1f73c515198685  |0.8214382913006505|1.0       |
|Windows 98                 |0.993063798390144 |1.0       |
|Windows 98                 |0.993063798390144 |1.0       |
|V1818CA                    |0.684273528029555 |1.0       |
|HLK-AL00                   |0.989309856

In [205]:
tar.groupBy('val','prediction').count().sort('count',ascending=False).show(50,truncate=False)

+-------------------------+----------+------+
|val                      |prediction|count |
+-------------------------+----------+------+
|Windows 98               |1.0       |418688|
|V1818CA                  |1.0       |33542 |
|HUAWEI                   |1.0       |26110 |
|V1818A                   |1.0       |17871 |
|Windows                  |1.0       |13055 |
|Windows 9                |1.0       |10595 |
|WindowsBaiduYunGuanJia   |1.0       |7753  |
|Xiaomi                   |1.0       |7301  |
|Windows NT               |1.0       |7047  |
|localhost                |1.0       |5062  |
|Windows                  |1.0       |3781  |
|Generic Android on mt5882|1.0       |3690  |
|virtual machine 2        |1.0       |3465  |
|X7                       |1.0       |2367  |
|Windows NT               |1.0       |2353  |
|M5                       |1.0       |2015  |
|Windows N                |1.0       |1767  |
|Windows NT 6             |1.0       |1651  |
|QCA4002                  |1.0    

# ML - RF

In [285]:
from pyspark.sql.functions import *

d1 = spark.read.parquet('/data/user/hive/warehouse/ian/feature/unrecognized/*')
d2 = spark.read.parquet('/data/user/hive/warehouse/ian/feature/recognized/*')
d2 = d2.sample(0.20,seed=123)

df = d1.union(d2)

from pyspark.ml.feature import MinMaxScaler,StandardScaler,VectorAssembler,StringIndexer

vec = VectorAssembler(inputCols=['f1','f2','f3','f4','f5','f6','f7'],outputCol='features')
vec_ = vec.transform(df)
#minmax= MinMaxScaler(inputCol='rawfeatures',outputCol='features')
#dd = minmax.fit(vec_)
#trainset = dd.transform(vec_).select('features','label')


In [286]:
d1.count(),d1.distinct().count(),d2.count(),d2.distinct().count(),df.count(),df.distinct().count()

(13366199, 13366199, 23718694, 23718694, 37084893, 37084893)

# baseline 

In [287]:
23718694/37084893

0.6395783318021169

In [234]:
vec_.show(10,truncate=False)

+---------------+-----+----+---+---+----+---+---+---+-----------------------------+
|val            |label|f1  |f2 |f3 |f4  |f5 |f6 |f7 |features                     |
+---------------+-----+----+---+---+----+---+---+---+-----------------------------+
|panquan        |0.0  |7.0 |0.0|0.0|7.0 |0.0|0.0|0.0|(7,[0,3],[7.0,7.0])          |
|VecTor         |0.0  |6.0 |1.0|0.0|4.0 |2.0|0.0|0.0|[6.0,1.0,0.0,4.0,2.0,0.0,0.0]|
|zhuping        |0.0  |7.0 |0.0|0.0|7.0 |0.0|0.0|0.0|(7,[0,3],[7.0,7.0])          |
|bishoujo       |0.0  |8.0 |0.0|0.0|8.0 |0.0|0.0|0.0|(7,[0,3],[8.0,8.0])          |
|rouko          |0.0  |5.0 |0.0|0.0|5.0 |0.0|0.0|0.0|(7,[0,3],[5.0,5.0])          |
|daniaoyudadiaoo|0.0  |15.0|0.0|0.0|15.0|0.0|0.0|0.0|(7,[0,3],[15.0,15.0])        |
|daikoikoi      |0.0  |9.0 |0.0|0.0|9.0 |0.0|0.0|0.0|(7,[0,3],[9.0,9.0])          |
|Daisy          |0.0  |5.0 |1.0|0.0|4.0 |1.0|0.0|0.0|[5.0,1.0,0.0,4.0,1.0,0.0,0.0]|
|8848           |0.0  |4.0 |0.0|4.0|0.0 |0.0|0.0|0.0|(7,[0,2],[4.0,4.0])    

In [235]:
trainDF, testDF = vec_.randomSplit([0.7,0.3],seed=0)

#stringIndexer = StringIndexer(inputCol='label',outputCol='indexed')
#si_model = stringIndexer.fit(trainset)
#td = si_model.transform(trainset)

from pyspark.ml.classification import RandomForestClassifier
#rf =RandomForestClassifier(maxDepth=3)
#rf_model = rf.fit(td)

In [241]:
rf =RandomForestClassifier(maxDepth=3)
rf_model = rf.fit(trainDF)

In [ ]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator,ParamGridBuilder
import numpy as np

lr = LogisticRegression(maxIter=10)

paramGrid_lr = ParamGridBuilder()\
       .addGrid(lr.regParam,np.linspace(0.3,0.01,10))\
       .addGrid(lr.elasticNetParam,np.linspace(0.3,0.8,6))\
       .build()

crossval_lr = CrossValidator(estimator=lr,
                            estimatorParamMaps=paramGrid_lr,
                            evaluator=BinaryClassificationEvaluator(),
                            numFolds=5)

cvModel_lr = crossval_lr.fit(trainDF)
best_model_lr = cvModel_lr.bestModel.summary
best_model_lr.predictions.columns

In [250]:
rf_model.transform(trainDF).groupBy('label','prediction').count().show(truncate=False)

+-----+----------+--------+
|label|prediction|count   |
+-----+----------+--------+
|1.0  |0.0       |942080  |
|0.0  |0.0       |8694283 |
|1.0  |1.0       |15666039|
|0.0  |1.0       |660875  |
+-----+----------+--------+



In [252]:
(8694283 +15666039)/(942080 +8694283 +15666039+660875)

0.9382606825787053

In [253]:
rf_model.transform(testDF).groupBy('label','prediction').count().show(truncate=False)

+-----+----------+-------+
|label|prediction|count  |
+-----+----------+-------+
|1.0  |0.0       |403106 |
|0.0  |0.0       |3727554|
|1.0  |1.0       |6710375|
|0.0  |1.0       |283487 |
+-----+----------+-------+



In [243]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
my_eval_lr = BinaryClassificationEvaluator(
    rawPredictionCol='prediction', labelCol='label', metricName='areaUnderROC')
my_eval_lr.evaluate(rf.predictions)

AttributeError: 'RandomForestClassifier' object has no attribute 'predictions'

In [ ]:
train_fit_lr = best_model_lr.predictions.select('label', 'prediction')
train_fit_lr.groupBy('label', 'prediction').count().show()

# feature_pipeline

In [261]:
from pyspark.sql.functions import *
d = spark.read.csv('/user/maxnet/database/sig.db/data_visual_unknown/*',sep='\x01')
d1 = d.select('_c2').withColumnRenamed('_c2','val').distinct().dropna()

d1 = d1.withColumn('f1',length(col('val')))

d1 = d1.withColumn('f2',when(d1.val.startswith('A')|d1.val.startswith('B')|d1.val.startswith('C')\
                             |d1.val.startswith('D')|d1.val.startswith('E')|d1.val.startswith('F')\
                             |d1.val.startswith('G')|d1.val.startswith('H')|d1.val.startswith('I')\
                             |d1.val.startswith('J')|d1.val.startswith('K')|d1.val.startswith('L')\
                             |d1.val.startswith('M')|d1.val.startswith('N')|d1.val.startswith('O')\
                             |d1.val.startswith('P')|d1.val.startswith('Q')|d1.val.startswith('R')\
                             |d1.val.startswith('S')|d1.val.startswith('T')|d1.val.startswith('U')\
                             |d1.val.startswith('V')|d1.val.startswith('W')|d1.val.startswith('X')\
                             |d1.val.startswith('Y')|d1.val.startswith('Z'),1).otherwise(0))

import re

num_regex = re.compile(r'[0-9]') #数字
xiaoxiezimu_regex = re.compile(r'[a-z]')#小写字母
daxiezimu_regex = re.compile(r'[A-Z]') #大写字母 
#hanzi_regex = re.compile(r'[\u4E00-\u9FA5]')#汉字

from pyspark.sql.functions import udf
num = udf(lambda x: len(num_regex.findall(x)))
xiaoxie = udf(lambda x: len(xiaoxiezimu_regex.findall(x)))
daxie = udf(lambda x: len(daxiezimu_regex.findall(x)))

d1 = d1.withColumn('f3',num('val'))
d1 = d1.withColumn('f4',xiaoxie('val'))
d1 = d1.withColumn('f5',daxie('val'))

# 特征字符串长度 f1
# 首字母是否大写 f2
# 数字字符数量   f3
# 小写字母数量   f4
# 大写字母数量   f5
# 特殊符号-_:的数量 f6
# 空格的数量 f7


# 统计下划线个数
def xiahuaxian_count(s):
    xiahuaxian_counts=0
    for c in s:
        xiahuaxian_split_list = c.split('_')
        xiahuaxian_counts += len(xiahuaxian_split_list) - 1
    return xiahuaxian_counts

# 统计中划线个数
def zhonghuaxian_count(s):
    zhonghuaxian_counts=0
    for c in s:
        zhonghuaxian_split_list = c.split('-')
        zhonghuaxian_counts += len(zhonghuaxian_split_list) - 1
    return zhonghuaxian_counts

# 统计冒号个数
def maohao_count(s):
    maohao_counts=0
    for c in s:
        maohao_split_list = c.split(':')
        maohao_counts += len(maohao_split_list) - 1
    return maohao_counts

def teshu_count(s):
    teshu_counts=0
    a_counts=0
    b_counts=0
    c_counts=0
    for c in s:
        a_split_list = c.split('_')
        a_counts += len(a_split_list) - 1
        
        b_split_list = c.split('-')
        b_counts += len(b_split_list) - 1
        
        c_split_list = c.split(':')
        c_counts += len(c_split_list) - 1
        
        teshu_counts = a_counts + b_counts + c_counts
    return teshu_counts
        

# 统计空格个数
def space_count(s):
    space_counts=0
    for c in s:
        space_split_list = c.split(' ')
        space_counts += len(space_split_list) - 1
    return space_counts

teshu = udf(lambda x: teshu_count(x))
kongge = udf(lambda x: space_count(x))


d1 = d1.withColumn('f6',teshu('val'))
d1 = d1.withColumn('f7',kongge('val'))


d1 = d1.select('val',col('f1').cast('float'),col('f2').cast('float'),col('f3').cast('float'),col('f4').cast('float'),col('f5').cast('float'),col('f6').cast('float'),col('f7').cast('float'))

d1.show(truncate=False)
#ddd = d1.repartition(1)

#ddd.write.mode('overwrite').parquet('hdfs:///data/user/hive/warehouse/ian/feature/unrecognized/',compression='gzip')

+-------------------------------+----+---+----+----+----+---+---+
|val                            |f1  |f2 |f3  |f4  |f5  |f6 |f7 |
+-------------------------------+----+---+----+----+----+---+---+
|U94JW11SOGYVGDE                |15.0|1.0|4.0 |0.0 |11.0|0.0|0.0|
|BRW0C96E67F9881                |15.0|1.0|9.0 |0.0 |6.0 |0.0|0.0|
|LCDN-20190619YJ                |15.0|1.0|8.0 |0.0 |6.0 |1.0|0.0|
|MV17741370                     |10.0|1.0|8.0 |0.0 |2.0 |0.0|0.0|
|05af9271958ff346b26a47b3a5ef7a7|31.0|0.0|20.0|11.0|0.0 |0.0|0.0|
|ZTE24:58:6e:84:98:38           |20.0|1.0|11.0|1.0 |3.0 |5.0|0.0|
|atadevice24:58:6e:8b:af:c0     |26.0|0.0|7.0 |14.0|0.0 |5.0|0.0|
|GM-T9+                         |6.0 |1.0|1.0 |0.0 |3.0 |1.0|0.0|
|D08185818                      |9.0 |1.0|8.0 |0.0 |1.0 |0.0|0.0|
|V2C                            |3.0 |1.0|1.0 |0.0 |2.0 |0.0|0.0|
|0e0118cfb5d5ed8ffaee177fc4dd7f0|31.0|0.0|14.0|17.0|0.0 |0.0|0.0|
|0a7575c942b1bfcf58e2c7e249f078b|31.0|0.0|19.0|12.0|0.0 |0.0|0.0|
|ZTE8c:68:

In [262]:
d1.count(),d1.distinct().count()

(5625, 5625)

In [264]:
from pyspark.ml.feature import MinMaxScaler,StandardScaler,VectorAssembler,StringIndexer

unknow = VectorAssembler(inputCols=['f1','f2','f3','f4','f5','f6','f7'],outputCol='features')
unknow = vec.transform(d1)

In [267]:
t = rf_model.transform(unknow)

In [269]:
t.dtypes

[('val', 'string'),
 ('f1', 'float'),
 ('f2', 'float'),
 ('f3', 'float'),
 ('f4', 'float'),
 ('f5', 'float'),
 ('f6', 'float'),
 ('f7', 'float'),
 ('features', 'vector'),
 ('rawPrediction', 'vector'),
 ('probability', 'vector'),
 ('prediction', 'double')]

In [270]:
pred = t.select('val',unlist('probability').alias('probability'),'prediction').filter(t.prediction == 1).filter(t.val != 'unknown').filter(t.val != 'empty').filter(t.val != 'NONE').filter(t.val != 'none').filter(t.val != 'N/A').filter(t.val != 'normal').filter(t.val != 'anonymous').filter(t.val != 'null')

In [271]:
pred.show(truncate=False)

+---------------------------------------+------------------+----------+
|val                                    |probability       |prediction|
+---------------------------------------+------------------+----------+
|LCDN-20190619YJ                        |0.5142811462878983|1.0       |
|atadevice24:58:6e:8b:af:c0             |0.9907474363076645|1.0       |
|AUTOBVT-61AGAV1                        |0.5288482623222461|1.0       |
|cm7221_u11312_h7S68_M7S                |0.8136834078626346|1.0       |
|BF-20190327BFCE                        |0.5142811462878983|1.0       |
|Generic Android on mt5882              |0.6623334322088122|1.0       |
|Netopia-3000/157090527312              |0.8863456558686088|1.0       |
|Generic+Android+on+mt5882              |0.6623334322088122|1.0       |
|XCB-20160304VKM                        |0.5142811462878983|1.0       |
|AliFunUI_Philips_43PFF5081T3_MT5507_N40|0.7529748553537993|1.0       |
|CY-20181015NEXC                        |0.5142811462878983|1.0 

In [275]:
pred.sort('probability',ascending=False).show(100,truncate=False)

+---------------------------------------+------------------+----------+
|val                                    |probability       |prediction|
+---------------------------------------+------------------+----------+
|atadevice24:58:6e:8b:84:8c             |0.9907474363076645|1.0       |
|atadevice24:58:6e:8b:ed:e4             |0.9907474363076645|1.0       |
|atadevice24:58:6e:8b:88:dc             |0.9907474363076645|1.0       |
|nova_5-dfd25331d594626a                |0.9907474363076645|1.0       |
|atadevice24:58:6e:8b:fd:20             |0.9907474363076645|1.0       |
|atadevice24:58:6e:8b:9e:f0             |0.9907474363076645|1.0       |
|atadevice24:58:6e:8b:ea:70             |0.9907474363076645|1.0       |
|atadevice24:58:6e:8b:f6:90             |0.9907474363076645|1.0       |
|nova_5-b1c53667c38d34a2                |0.9907474363076645|1.0       |
|nova_5-3cc9ccabbf8780f6                |0.9907474363076645|1.0       |
|atadevice24:58:6e:8b:eb:fc             |0.9907474363076645|1.0 

In [293]:
# rf_model.save('hdfs:///data/user/hive/warehouse/ian/model/model3')
# m3 = rf_model.load('hdfs:///data/user/hive/warehouse/ian/model/model3')

In [289]:
t.select('val',unlist('probability').alias('probability'),'prediction').groupBy('prediction').count().show(truncate=False)

+----------+-----+
|prediction|count|
+----------+-----+
|0.0       |5279 |
|1.0       |346  |
+----------+-----+



In [296]:
#pred_hdfs = pred.repartition(1)

#pred_hdfs.write.mode('overwrite').parquet('hdfs:///data/user/hive/warehouse/ian/prediction/p1',compression='gzip')